In [3]:
import json

with open("operations_administratives_maroc.json", "r", encoding="utf-8") as f:
    data = json.load(f)
data

{"Documents d'identité": [{'id': 1,
   'operation': "Première demande CNIE (Carte Nationale d'Identité Électronique)",
   'categorie': "Documents d'identité",
   'documents_requis': ['Formulaire de pré-demande imprimé',
    "4 photos d'identité récentes conformes OACI",
    "Copie intégrale d'acte de naissance (en arabe/latin)",
    'Certificat de résidence',
    'Original du livret de famille ou copie selon cas'],
   'nombre_documents': 5,
   'autorite_competente': "Ministère de l'Intérieur / DGSN",
   'source_officielle': 'https://www.cnie.ma',
   'frais_estimés': 'Gratuit',
   'delai_moyen': '2 à 8 semaines',
   'format_sortie': 'Carte physique'},
  {'id': 2,
   'operation': 'Renouvellement CNIE',
   'categorie': "Documents d'identité",
   'documents_requis': ['Ancienne CNIE',
    'Formulaire de renouvellement',
    "2 photos d'identité",
    "Justificatif de changement d'état civil si nécessaire"],
   'nombre_documents': 4,
   'autorite_competente': "Ministère de l'Intérieur / DGSN

In [8]:
chunks = []

for category, docs in data.items():
    for doc in docs:
        text = f"""
ID: {doc['id']}
Opération: {doc['operation']}
Catégorie: {doc['categorie']}
Documents requis: {', '.join(doc['documents_requis'])}
Nombre de documents: {doc['nombre_documents']}
Autorité compétente: {doc['autorite_competente']}
Source officielle: {doc['source_officielle']}
Frais estimés: {doc['frais_estimés']}
Délai moyen: {doc['delai_moyen']}
Format de sortie: {doc['format_sortie']}
"""
        chunks.append({
            "id": doc['id'],
            "text": text,
            "categorie": doc['categorie']
        })
chunks[0]

{'id': 1,
 'text': "\nID: 1\nOpération: Première demande CNIE (Carte Nationale d'Identité Électronique)\nCatégorie: Documents d'identité\nDocuments requis: Formulaire de pré-demande imprimé, 4 photos d'identité récentes conformes OACI, Copie intégrale d'acte de naissance (en arabe/latin), Certificat de résidence, Original du livret de famille ou copie selon cas\nNombre de documents: 5\nAutorité compétente: Ministère de l'Intérieur / DGSN\nSource officielle: https://www.cnie.ma\nFrais estimés: Gratuit\nDélai moyen: 2 à 8 semaines\nFormat de sortie: Carte physique\n",
 'categorie': "Documents d'identité"}

In [20]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI()

def get_embedding(text: str):
    response = client.embeddings.create(
        model="text-embedding-3-small",  # ou text-embedding-3-large
        input=text
    )
    return response.data[0].embedding
embeddings_list = []

for chunk in chunks:
    embeddings_list.append({
        "id": chunk["id"],
        "categorie": chunk["categorie"],
        "text":chunk["text"],
        "vector": get_embedding(chunk["text"])
    })

In [21]:
print(embeddings_list[0])  # exemple de sortie pour le premier chunk

{'id': 1, 'categorie': "Documents d'identité", 'text': "\nID: 1\nOpération: Première demande CNIE (Carte Nationale d'Identité Électronique)\nCatégorie: Documents d'identité\nDocuments requis: Formulaire de pré-demande imprimé, 4 photos d'identité récentes conformes OACI, Copie intégrale d'acte de naissance (en arabe/latin), Certificat de résidence, Original du livret de famille ou copie selon cas\nNombre de documents: 5\nAutorité compétente: Ministère de l'Intérieur / DGSN\nSource officielle: https://www.cnie.ma\nFrais estimés: Gratuit\nDélai moyen: 2 à 8 semaines\nFormat de sortie: Carte physique\n", 'vector': [0.01624212972819805, 0.01262169610708952, 0.008104457519948483, -0.02989349141716957, 0.0020067389123141766, 0.0031360487919300795, -0.014537094160914421, 0.034432873129844666, -0.04320163279771805, 0.03638148680329323, 0.03303784504532814, -0.0271920058876276, -0.017803234979510307, -0.02486695535480976, -0.00736265629529953, 0.06638569384813309, -0.016740355640649796, 0.01305

In [22]:
prompt = "Comment je peux renouveller ma CNIE ?"
prompt_embedding = get_embedding(prompt)

In [23]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """
    Calcule la similarité cosinus entre deux vecteurs
    """
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    # Produit scalaire / (norme de vec1 * norme de vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [29]:
# Calculer la similarité entre le prompt et tous les chunks
similarities = []
for item in embeddings_list:
    sim = cosine_similarity(prompt_embedding, item["vector"])
    similarities.append({
        "id": item["id"],
        "text": item["text"],
        "similarity": sim
    })

# Trier par similarité décroissante et récupérer les 5 premiers
top_3 = sorted(similarities, key=lambda x: x["similarity"], reverse=True)[:3]

# Affichage
for idx, item in enumerate(top_3, 1):
    print(f"{idx}. ID: {item['id']}, Catégorie: {item['text']}, Similarité: {item['similarity']:.4f}")

1. ID: 2, Catégorie: 
ID: 2
Opération: Renouvellement CNIE
Catégorie: Documents d'identité
Documents requis: Ancienne CNIE, Formulaire de renouvellement, 2 photos d'identité, Justificatif de changement d'état civil si nécessaire
Nombre de documents: 4
Autorité compétente: Ministère de l'Intérieur / DGSN
Source officielle: https://www.cnie.ma
Frais estimés: Gratuit
Délai moyen: 1 à 4 semaines
Format de sortie: Carte physique
, Similarité: 0.6628
2. ID: 1, Catégorie: 
ID: 1
Opération: Première demande CNIE (Carte Nationale d'Identité Électronique)
Catégorie: Documents d'identité
Documents requis: Formulaire de pré-demande imprimé, 4 photos d'identité récentes conformes OACI, Copie intégrale d'acte de naissance (en arabe/latin), Certificat de résidence, Original du livret de famille ou copie selon cas
Nombre de documents: 5
Autorité compétente: Ministère de l'Intérieur / DGSN
Source officielle: https://www.cnie.ma
Frais estimés: Gratuit
Délai moyen: 2 à 8 semaines
Format de sortie: Carte 

In [30]:
print(top_3[0])

{'id': 2, 'text': "\nID: 2\nOpération: Renouvellement CNIE\nCatégorie: Documents d'identité\nDocuments requis: Ancienne CNIE, Formulaire de renouvellement, 2 photos d'identité, Justificatif de changement d'état civil si nécessaire\nNombre de documents: 4\nAutorité compétente: Ministère de l'Intérieur / DGSN\nSource officielle: https://www.cnie.ma\nFrais estimés: Gratuit\nDélai moyen: 1 à 4 semaines\nFormat de sortie: Carte physique\n", 'similarity': 0.6627513389804219}


In [31]:


# Construire un contexte à partir des chunks
context = "\n\n".join([f"{chunk['text']}" for chunk in top_3])

# Prompt final pour le LLM
final_prompt = f"""
Tu es un assistant administratif. Utilise les informations suivantes pour répondre à la question de l'utilisateur :

Contexte :
{context}

Question de l'utilisateur :
{prompt}

Répond de manière claire et concise.
"""

# Appel au LLM pour générer la réponse
response = client.chat.completions.create(
    model="gpt-4o-mini",  # ou gpt-4o, gpt-4o-mini, gpt-4-turbo
    messages=[
        {"role": "system", "content": "Tu es un assistant administratif expert."},
        {"role": "user", "content": final_prompt}
    ],
    temperature=0.2
)

# Récupérer la réponse
answer = response.choices[0].message.content
print(answer)


Pour renouveler votre Carte Nationale d'Identité Électronique (CNIE), vous devez suivre les étapes suivantes :

1. **Documents requis** :
   - Ancienne CNIE
   - Formulaire de renouvellement
   - 2 photos d'identité
   - Justificatif de changement d'état civil si nécessaire

2. **Nombre de documents** : 4

3. **Autorité compétente** : Ministère de l'Intérieur / DGSN

4. **Frais estimés** : Gratuit

5. **Délai moyen** : 1 à 4 semaines

6. **Format de sortie** : Carte physique

Pour plus d'informations, vous pouvez consulter la source officielle : [cnie.ma](https://www.cnie.ma).
